In [18]:
import pandas as pd
import random
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [19]:
 #Step 1: Generate Synthetic Data 
scenarios = [
    {"context": "Customer is asking about order tracking.", "agent_action": "Provide tracking details."},
    {"context": "Customer wants to know the delivery status.", "agent_action": "Check and inform about the delivery status."},
    {"context": "Customer wishes to cancel an order.", "agent_action": "Process the order cancellation."},
    {"context": "Customer asks for order replacement.", "agent_action": "Initiate the order replacement process."},
    {"context": "Customer inquires about returning a product.", "agent_action": "Guide the customer through the return process."},
    {"context": "Customer has a complaint about a product's quality.", "agent_action": "Address the complaint and offer a solution."},
    {"context": "Customer asks for a product recommendation.", "agent_action": "Provide product recommendations."},
    {"context": "Customer needs help with product setup.", "agent_action": "Assist with the product setup instructions."},
    {"context": "Customer is checking on warranty coverage.", "agent_action": "Provide warranty coverage information."},
    {"context": "Customer wants to update their contact information.", "agent_action": "Update the customer's contact details."},
    {"context": "Customer inquires about payment options.", "agent_action": "Explain the payment options."},
    {"context": "Customer is experiencing a technical issue.", "agent_action": "Offer troubleshooting steps."},
    {"context": "Customer wants to know about loyalty program benefits.", "agent_action": "Describe the loyalty program benefits."},
    {"context": "Customer asks about product recycling.", "agent_action": "Provide recycling information."},
    {"context": "Customer is inquiring about how to use a voucher.", "agent_action": "Explain the process of redeeming a voucher."},
    {"context": "Customer expresses dissatisfaction with a recent purchase.", "agent_action": "Acknowledge the issue and discuss possible resolutions."},
    {"context": "Customer is confused about a policy.", "agent_action": "Clarify the policy details for the customer."},
    {"context": "Customer received a defective product.", "agent_action": "Apologize and initiate a product exchange or return."},
    {"context": "Customer is asking about international shipping options.", "agent_action": "Provide information on international shipping policies and costs."},
    {"context": "Customer wants to change their subscription plan.", "agent_action": "Guide the customer through the process of changing their subscription."},
    {"context": "Customer needs assistance with resetting their password.", "agent_action": "Provide step-by-step instructions for resetting the password."},
    {"context": "Customer inquires about the availability of a product.", "agent_action": "Check and inform the customer about the product's availability."},
    {"context": "Customer is asking for an extension on a payment deadline.", "agent_action": "Discuss the possibility and process of extending the payment deadline."},
    {"context": "Customer wants to provide feedback on a service.", "agent_action": "Thank the customer for their feedback and guide them on how to submit it."},
]


In [20]:
data = set()
for _ in range(1000):  # Generate 1000 unique samples
    scenario = random.choice(scenarios)
    customer_message = f"Customer: {scenario['context']} Agent: {scenario['agent_action']}"
    data.add(customer_message)

data

{'Customer: Customer asks about product recycling. Agent: Provide recycling information.',
 'Customer: Customer asks for a product recommendation. Agent: Provide product recommendations.',
 'Customer: Customer asks for order replacement. Agent: Initiate the order replacement process.',
 'Customer: Customer expresses dissatisfaction with a recent purchase. Agent: Acknowledge the issue and discuss possible resolutions.',
 "Customer: Customer has a complaint about a product's quality. Agent: Address the complaint and offer a solution.",
 'Customer: Customer inquires about payment options. Agent: Explain the payment options.',
 'Customer: Customer inquires about returning a product. Agent: Guide the customer through the return process.',
 "Customer: Customer inquires about the availability of a product. Agent: Check and inform the customer about the product's availability.",
 'Customer: Customer is asking about international shipping options. Agent: Provide information on international shi

In [21]:
# Save the unique data to a text file
with open('customer_service_data.txt', 'w') as f:
    for item in data:
        f.write("%s\n" % item)

In [22]:
# Step 2: Fine-Tune the GPT-2 Model
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [23]:
train_dataset = TextDataset(
  tokenizer=tokenizer,
  file_path="customer_service_data.txt",
  block_size=128
)

c:\Users\saket.singh1\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [24]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)  #False

In [25]:
training_args = TrainingArguments(
    output_dir="./gpt2-customer-service",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

trainer.train()

trainer.save_model("./gpt2-customer-service")

100%|██████████| 175/175 [11:39<00:00,  4.00s/it]


{'train_runtime': 699.851, 'train_samples_per_second': 1.0, 'train_steps_per_second': 0.25, 'train_loss': 0.18299564906529017, 'epoch': 5.0}


In [27]:
import random
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Ensure the model and tokenizer paths are correct
model_path = "./gpt2-customer-service"
tokenizer_path = "gpt2"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def get_random_dialogue(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            lines = [line.strip() for line in file if line.strip()]
            return random.choice(lines)
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return ""

def generate_next_sentence(dialogue):
    prompt = f"Given the following conversation, predict the next sentence:\n\n{dialogue}\n\nNext sentence:"
    inputs = tokenizer(prompt, return_tensors='pt')
    #output = model.generate(inputs["input_ids"], max_length=inputs["input_ids"].shape[1] + 50, pad_token_id=tokenizer.eos_token_id)
    output = model.generate(
        inputs["input_ids"],
        max_length=inputs["input_ids"].shape[1]+ 15,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
        temperature=0.9, #0.7,
        top_k=50,
        top_p=0.95
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return prompt, decoded_output

# Get a random dialogue and generate the next sentence
dialogue = get_random_dialogue('customer_service_data.txt')
if dialogue:
    prompt, output = generate_next_sentence(dialogue)
    print('-' * 100)
    print(f'INPUT PROMPT:\n{prompt}')
    print('-' * 100)
    print(f'MODEL GENERATION - NEXT SENTENCE PREDICTION:\n{output}')


c:\Users\saket.singh1\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\saket.singh1\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Given the following conversation, predict the next sentence:

Customer: Customer is asking about order tracking. Agent: Provide tracking details.

Next sentence:
----------------------------------------------------------------------------------------------------
MODEL GENERATION - NEXT SENTENCE PREDICTION:
Given the following conversation, predict the next sentence:

Customer: Customer is asking about order tracking. Agent: Provide tracking details.

Next sentence: Customer asks for order replacement. Agent: Initiate the order replacement process.


In [28]:
# Modified function to generate the next sentence based on a given dialogue
def generate_next_sentence_from_specific_input(customer_input):
    dialogue = f"Customer: {customer_input}"
    prompt = f"Given the following conversation, predict the next sentence:\n\n{dialogue}\n\nNext sentence:"
    inputs = tokenizer(prompt, return_tensors='pt')
    output = model.generate(
        inputs["input_ids"],
        max_length=inputs["input_ids"].shape[1] + 25,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
        temperature=0.6,
        top_k=50,
        top_p=0.95
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    return prompt, decoded_output

# Test the model with a specific customer sentence
customer_input = "I want to cancel my order"
prompt, output = generate_next_sentence_from_specific_input(customer_input)
print('-' * 100)
print(f'INPUT PROMPT:\n{prompt}')
print('-' * 100)
print(f'MODEL GENERATION - NEXT SENTENCE PREDICTION:\n{output}')


c:\Users\saket.singh1\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Given the following conversation, predict the next sentence:

Customer: I want to cancel my order

Next sentence:
----------------------------------------------------------------------------------------------------
MODEL GENERATION - NEXT SENTENCE PREDICTION:
Given the following conversation, predict the next sentence:

Customer: I want to cancel my order

Next sentence: Process the order cancellation.
Customer: Customer asks for order replacement. Agent: Initiate the order replacement process.
Customer


In [17]:
#trial with a different prompt

from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2-customer-service")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

prompt = """
Context: This is a customer service conversation. The goal is to continue the dialogue in a coherent and contextually relevant manner.

Dialogue:
Customer: "Customer wants to provide feedback on a service."
Agent: "Thank the customer for their feedback and guide them on how to submit it."

Predict the next sentence in the conversation. Begin with the customer's response.

Next sentence:
"""

inputs = tokenizer(prompt, return_tensors='pt')

# Adjusting the generation parameters to encourage diversity
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_length=inputs["input_ids"].shape[1] + 50,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.9,  # Adjust this for more randomness
        top_k=50,  # Top-k sampling
        top_p=0.95,  # Top-p (nucleus) sampling
        num_return_sequences=1,
    )[0],
    skip_special_tokens=True
)

generated_continuation = output.split("Next sentence:")[1].strip()
print("Generated continuation:", generated_continuation)



Generated continuation: Customer: Customer asks for order replacement. Agent: Initiate the order replacement process.
Customer: Customer asks for order replacement. Agent: Initiate the order replacement process.
Customer: Customer asks for order replacement. Agent: Initiate the order
